In [1]:
from pathlib import Path
import re
import time
from datetime import datetime
from dateutil import relativedelta

import numpy as np
import pandas as pd
import toad

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import config
from util.db_helper import SQLite
from util.data_helper import Data
from projects.home_credit.features.ft_bureau_balance import FTBBalance

%load_ext autoreload
%autoreload 2

In [2]:
db_path = Path(config.ROOT_DIR, 'data', 'home_credit_default_risk.db')
SQLite.get_conn(db_path)

In [ ]:
SELECT B.SK_ID_CURR, BB.*
    FROM bureau B
    LEFT JOIN bureau_balance BB
    ON B.SK_ID_BUREAU = BB.SK_ID_BUREAU
    GROUP BY 

In [8]:
sql = """
    WITH 
    balance_tab AS (
        SELECT SK_ID_BUREAU, MAX(MONTHS_BALANCE) AS MONTHS_BALANCE
        FROM bureau_balance
        GROUP BY SK_ID_BUREAU
    ),
    recent_balance_tab AS (
        SELECT U.SK_ID_BUREAU, U.MONTHS_BALANCE, BB.STATUS
        FROM balance_tab U
        LEFT JOIN bureau_balance BB
        ON U.SK_ID_BUREAU=BB.SK_ID_BUREAU AND U.MONTHS_BALANCE = BB.MONTHS_BALANCE
    )
    
    SELECT B.SK_ID_CURR, B.SK_ID_BUREAU, B.DAYS_CREDIT, 
           RB.MONTHS_BALANCE, RB.STATUS
    FROM bureau B
    LEFT JOIN recent_balance_tab RB
    ON B.SK_ID_BUREAU=RB.SK_ID_BUREAU
"""
df_bbalance = SQLite.query(sql)

In [ ]:
df_bbalance.head(3)

In [14]:
Data.dump('df_bbalance', df_bbalance)

In [18]:
df_bbalance = Data.load('df_bbalance')

In [27]:
df_bbalance = FTBBalance.preprocess(df_bbalance)

In [16]:
df_dim = pd.DataFrame({'SK_ID_CURR': df_bbalance['SK_ID_CURR'].unique()})

In [35]:
df_ft = FTBBalance.get_feature_per_period(df_dim, df_bbalance, 1200)

In [36]:
df_ft

,SK_ID_CURR,ft_bb_no_history_cnt,bureau_balance_dpd_0,ft_bb_bureau_balance_dpd_1_30_cnt,ft_bb_bureau_balance_dpd_30plus_cnt
0,215354,8.0,0.0,0.0,0.0
1,162297,4.0,0.0,0.0,0.0
2,402440,1.0,0.0,0.0,0.0
3,238881,6.0,0.0,0.0,0.0
4,222183,7.0,0.0,0.0,0.0
...,...,...,...,...,...
305806,207190,0.0,0.0,0.0,0.0
305807,324956,0.0,0.0,0.0,0.0
305808,448157,NaN,NaN,NaN,NaN
305809,345866,0.0,0.0,0.0,0.0


In [37]:
df_ft['ft_bb_bureau_balance_dpd_30plus_cnt'].value_counts(dropna=False)

0.0     275996
NaN      28938
1.0        739
2.0         91
3.0         31
4.0         10
5.0          3
7.0          1
10.0         1
8.0          1
Name: ft_bb_bureau_balance_dpd_30plus_cnt, dtype: int64

In [38]:
df_ft[df_ft['ft_bb_bureau_balance_dpd_30plus_cnt']>8]

,SK_ID_CURR,ft_bb_no_history_cnt,bureau_balance_dpd_0,ft_bb_bureau_balance_dpd_1_30_cnt,ft_bb_bureau_balance_dpd_30plus_cnt
129472,126228,0.0,0.0,0.0,10.0


In [39]:
df_bbalance.loc[df_bbalance['SK_ID_CURR']==126228]

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,MONTHS_BALANCE,STATUS,no_bureau_balance_history,bureau_balance_dpd_0,bureau_balance_dpd_1_30,bureau_balance_dpd_30plus
564139,126228,5692226,-268,0.0,5,0,0,0,1
626676,126228,5692230,-268,0.0,5,0,0,0,1
626677,126228,5692231,-268,0.0,5,0,0,0,1
626672,126228,5692225,-334,-2.0,5,0,0,0,1
626675,126228,5692229,-334,-2.0,5,0,0,0,1
626674,126228,5692228,-359,-2.0,5,0,0,0,1
626671,126228,5692224,-386,-3.0,5,0,0,0,1
564138,126228,5692222,-703,-14.0,5,0,0,0,1
626670,126228,5692223,-703,-14.0,5,0,0,0,1
626673,126228,5692227,-933,-21.0,5,0,0,0,1
